### MySQL 접속용 샘플 프로그램
- pymysql 설치할 것

##### 1. 접속 정보가 들어 있는 파일 읽기

In [1]:
import json
with open('./mysql.json') as f:
    config_str = f.read()
config = json.loads(config_str)
config

{'host': 'localhost',
 'user': 'ysuser',
 'password': 'yspass',
 'database': 'world',
 'port': 3306}

##### 2. MySQL에 접속

In [3]:
import pymysql
conn = pymysql.connect(
    host=config['host'], user=config['user'], password=config['password'],
    database=config['database'], port=config['port']
)
# conn = pymysql.connect(**config) 과 같이 사용할 수 있음

##### 3. MySQL Access

- city table에서 10개 데이터 읽기

In [4]:
cur = conn.cursor()
sql = "SELECT * FROM city LIMIT 10;"
cur.execute(sql)
rows = cur.fetchall()
cur.close()

for row in rows:
    print(row)

(1, 'Kabul', 'AFG', 'Kabol', 1780000)
(2, 'Qandahar', 'AFG', 'Qandahar', 237500)
(3, 'Herat', 'AFG', 'Herat', 186800)
(4, 'Mazar-e-Sharif', 'AFG', 'Balkh', 127800)
(5, 'Amsterdam', 'NLD', 'Noord-Holland', 731200)
(6, 'Rotterdam', 'NLD', 'Zuid-Holland', 593321)
(7, 'Haag', 'NLD', 'Zuid-Holland', 440900)
(8, 'Utrecht', 'NLD', 'Utrecht', 234323)
(9, 'Eindhoven', 'NLD', 'Noord-Brabant', 201843)
(10, 'Tilburg', 'NLD', 'Noord-Brabant', 193238)


In [6]:
import pandas as pd
df = pd.DataFrame(rows, columns=['id','name','country_code','district','population'])
df.set_index('id', inplace=True)
df

,name,country_code,district,population
id,,,,
1,Kabul,AFG,Kabol,1780000
2,Qandahar,AFG,Qandahar,237500
3,Herat,AFG,Herat,186800
4,Mazar-e-Sharif,AFG,Balkh,127800
5,Amsterdam,NLD,Noord-Holland,731200
6,Rotterdam,NLD,Zuid-Holland,593321
7,Haag,NLD,Zuid-Holland,440900
8,Utrecht,NLD,Utrecht,234323
9,Eindhoven,NLD,Noord-Brabant,201843


- 검색: 국가코드 KOR 의 인구수 Top 10

In [7]:
cur = conn.cursor()
sql = "SELECT * FROM city WHERE countrycode=%s ORDER BY population DESC LIMIT 10;"
cur.execute(sql, ('KOR', ))      # 파라메터는 반드시 튜플로 전달해야 함
rows = cur.fetchall()
cur.close()

In [8]:
for row in rows:
    print(row)

(2331, 'Seoul', 'KOR', 'Seoul', 10000000)
(5003, '대도시', 'KOR', 'Kyonggi', 8000000)
(2332, 'Pusan', 'KOR', 'Pusan', 3804522)
(2333, 'Inchon', 'KOR', 'Inchon', 2559424)
(2334, 'Taegu', 'KOR', 'Taegu', 2548568)
(2335, 'Taejon', 'KOR', 'Taejon', 1425835)
(2336, 'Kwangju', 'KOR', 'Kwangju', 1368341)
(2340, '수원', 'KOR', 'Kyonggi', 1200000)
(2337, 'Ulsan', 'KOR', 'Kyongsangnam', 1084891)
(2338, 'Songnam', 'KOR', 'Kyonggi', 869094)


In [16]:
cur = conn.cursor()
sql = "SELECT * FROM city WHERE name=%s;"
cur.execute(sql, ('수원', ))        # 파라메터는 반드시 튜플로 전달해야 함
row = cur.fetchone()                # 하나의 레코드
cur.close()
row

(2340, '수원', 'KOR', 'Kyonggi', 1200000)

- 데이터 추가(Insert)

In [9]:
cur = conn.cursor()
sql = "INSERT INTO city VALUES(DEFAULT, '자바','KOR','Kyonggi', 6000000);"
cur.execute(sql)
conn.commit()               # DB가 변화되는 SQL(Insert, Update, Delete)에서는 필수
cur.close()

In [10]:
params = ('노드', 'KOR', 'Kyonggi', 6000000)

cur = conn.cursor()
sql = "INSERT INTO city VALUES(DEFAULT, %s, %s, %s, %s);"
cur.execute(sql, params)
conn.commit()               # DB가 변화되는 SQL(Insert, Update, Delete)에서는 필수
cur.close()

In [12]:
params_list = [
    ('HTML', 'KOR', 'Kyonggi', 7000000), ('CSS', 'KOR', 'Kyonggi', 7200000)
]
cur = conn.cursor()
sql = "INSERT INTO city VALUES(DEFAULT, %s, %s, %s, %s);"
# for params in params_list:
#    cur.execute(sql, params)
cur.executemany(sql, params_list)       # 몇건은 가능하나, 대량은 어려울 수 있음
conn.commit()
cur.close()

- 데이터 수정(Update)

In [13]:
cur = conn.cursor()
sql = "UPDATE city SET name='CSS3' WHERE id=5008;"
cur.execute(sql)
conn.commit()               # DB가 변화되는 SQL(Insert, Update, Delete)에서는 필수
cur.close()

In [14]:
params_list = [('HTML5', 5007), ('HTML5', 5009)]
cur = conn.cursor()
sql = "UPDATE city SET name=%s WHERE id=%s;"
cur.executemany(sql, params_list)
conn.commit() 
cur.close()

- 데이터 삭제(Delete)

In [15]:
cur = conn.cursor()
sql = "DELETE FROM city WHERE id >= %s;"
cur.execute(sql, (5000,))
conn.commit() 
cur.close()

##### 4. MySQL 접속 해제

In [17]:
conn.close()